# Writing out a USGSCSM ISD from a PDS3 Kaguya image

In [1]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

# Make a CSM sensor model
Requires TC1S2B0_01_02842S506E1942.img in data directory

In [2]:
fileName = 'data/TC1S2B0_01_02842S506E1942.img'
camera = knoten.csm.create_csm(fileName)

In [3]:
# Set the output location of the resulting .cub
cub_loc = 'data/TC1S2B0_01_02842S506E1942.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

# Ingest the image and spiceinit

In [4]:
# Set the output location of the resulting .cub
cub_loc = 'data/TC1S2B0_01_02842S506E1942.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

# Define a function that takes an image coordinate, projects it to a ground point using ISIS, then projects the result back into an image coordinate using USGSCSM and computes the difference between image coordinates.

In [5]:
def check_pixel(camera, cub, line, sample):
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    line_diff = pvl_output['GroundPoint']['Line'] - image_coord.line - .5
    sample_diff = pvl_output['GroundPoint']['Sample'] - image_coord.samp - .5
    return line_diff, sample_diff

# Get the total number of lines / samples

In [6]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

# Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [7]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for _, row in pixels_df.iterrows():
    row['line_diff'], row['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])

pixels_df

line  sample  line_diff  sample_diff
0     1.0     1.0   0.610291     1.079313
1     1.0  1744.0   0.610945     0.992453
2  4656.0     1.0  -1.548140     1.099806
3  4656.0  1744.0  -1.547574     0.963186
4  2328.0   872.0  -1.542982     1.032304